## **Notebook : Découverte de PyPhi et des principes d’IIT**

### **Sommaire**
1. [Introduction à l’IIT](#sec1)  
2. [Installation et importation de PyPhi](#sec2)  
3. [Création d’un réseau simple (Network)](#sec3)  
4. [Définition d’un Subsystem et calcul de \(\Phi\)](#sec4)  
5. [Concepts, mécanismes, et structure cause-effet (CES)](#sec5)  
6. [Exploration d’exemples avancés (macro, actual causation…)](#sec6)  
7. [Ressources complémentaires](#sec7)

---

<a id="sec1"></a>
## 1. Introduction à l’IIT

**Objectif** : l’**Integrated Information Theory** (IIT), introduite par Giulio Tononi et collaborateurs, propose une mesure de la conscience à partir de la quantité d’information intégrée par un système. En pratique :

- Un **système** (réseau de neurones, circuit logique, etc.) est représenté par une **Transition Probability Matrix (TPM)**.
- L’information intégrée est notée \(\Phi\) (Phi). PyPhi calcule \(\Phi\) pour des sous-systèmes et explore la structure causale (Cause-Effect Structure, ou *CES*).
- L’**objectif pédagogique** de ce notebook est de créer un petit réseau, de définir un état, et de calculer \(\Phi\) pour différents sous-ensembles (subsystèmes).

---

<a id="sec2"></a>
## 2. Installation et importation de PyPhi

> **Remarque** : PyPhi exige parfois une installation depuis la source ou via `pip`.  
> **Pré-requis** : Python 3.7+ (recommandé).

Utiliser conda create --name pyphi python=3.7 -y

Sélectionner l'environnement nouvellement créé comme noyau

In [53]:
# Dans un terminal ou dans un environnement conda :
! pip install pyphi


In [54]:
import pyphi

print("PyPhi version:", pyphi.__version__)

PyPhi version: 1.2.0


<a id="sec3"></a>
## 3. Création d’un réseau simple (Network)

### 3.1. Structure d’un réseau

Un **Network** dans PyPhi est défini par :
- Une **TPM** (`ExplicitTPM`) en forme de matrice multidimensionnelle ou un array 2D.
- Une **connectivity matrix** (CM), indiquant comment les nœuds sont connectés.

### 3.2. Exemple : un circuit logique

Nous allons créer un réseau (circuit) minimaliste à 3 nœuds (A, B, C) :

- **A** : passerelle logique, reçoit des entrées de B et C  
- **B** : un XOR des entrées de A et C  
- **C** : identique à B ou un AND/OR, selon l’exemple souhaité 

In [55]:
import numpy as np
import pyphi

# Exemple d'un petit réseau XOR
# 3 noeuds : A, B, C
# On utilise un exemple déjà inclus dans pyphi.examples

network = pyphi.examples.xor_network()

print("Nodes:", network.node_labels)
print("TPM shape:", network.tpm.shape)
print("Connectivity matrix:\n", network.cm)

Nodes: NodeLabels(('A', 'B', 'C'))
TPM shape: (2, 2, 2, 3)
Connectivity matrix:
 [[0 1 1]
 [1 0 1]
 [1 1 0]]


**Comment ça marche ?**  
- `pyphi.examples.xor_network()` renvoie un réseau (Network) de taille 3, où chaque nœud est un XOR à deux entrées (sans auto-connexion).  
- La **TPM** est stockée dans `network.tpm`, et chaque dimension correspond à un état binaire (`0/1`). La `TPM` est un tableau multidimensionnel de taille (2, 2, 2, 3), reflétant les états binaires pour 3 nœuds.
La matrice de connectivité montre les liens (XOR signifie que chaque nœud dépend des deux autres sans boucles sur soi-même).


---

### 3.3. Inspection de la TPM

Pour mieux comprendre le fonctionnement du réseau, vous pouvez examiner directement les valeurs de la matrice de transition (TPM). Celle-ci peut être soit en _state-by-state_ (SBS) ou _state-by-node_ (SBN) selon votre cas. PyPhi propose des utilitaires de conversion.


In [56]:
import pyphi.convert

sbn_tpm = network.tpm
print("TPM (state-by-node) shape:", sbn_tpm.shape)

# Pour visualiser un 'slice' ou un 'flatten' :
print("Premier 'slice' de la TPM (correspondant à un sous-état) :\n", sbn_tpm[0])

# Passer au format state-by-state pour mieux le parcourir
sbs_tpm = pyphi.convert.state_by_node2state_by_state(sbn_tpm)
print("\nTPM (state-by-state) shape:", sbs_tpm.shape)
print("Quelques lignes du TPM:\n", sbs_tpm[:4])


TPM (state-by-node) shape: (2, 2, 2, 3)
Premier 'slice' de la TPM (correspondant à un sous-état) :
 [[[0. 0. 0.]
  [1. 1. 0.]]

 [[1. 0. 1.]
  [0. 1. 1.]]]

TPM (state-by-state) shape: (8, 8)
Quelques lignes du TPM:
 [[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]]


<a id="sec4"></a>
## 4. Définition d’un Subsystem et calcul de \(\Phi\)

### 4.1. Sous-système (Subsystem)

Un **Subsystem** se définit en spécifiant :
1. Un **Network** ;
2. Un **état** du réseau (ex. `(0, 0, 0)` pour `A=0, B=0, C=0`) ;
3. Les indices de nœuds internes qu’on considère comme faisant partie du sous-système.

Ensuite, PyPhi peut calculer la valeur de \(\Phi\) pour ce sous-système.

In [57]:
# État de départ
state = (0, 0, 0)

# Créons un Subsystem pour l’ensemble des nœuds (0, 1, 2)
subsystem = pyphi.Subsystem(network, state, (0, 1, 2))

# Calcul de Phi
phi_value = pyphi.compute.phi(subsystem)

print("Φ pour le sous-système (0,1,2) à l’état (0,0,0) :", phi_value)

Φ pour le sous-système (0,1,2) à l’état (0,0,0) : 1.874999


### 4.2. States inaccessibles et validation
PyPhi, par défaut, peut émettre des erreurs (`StateUnreachableError`) si l’état que vous définissez n’est pas accessible depuis la TPM. Ci-dessous, on montre comment désactiver temporairement la validation pour s’assurer que PyPhi ne bloque pas le calcul.


In [58]:
# Configurer PyPhi pour ignorer les inaccessibilités d’états
pyphi.config.VALIDATE_SUBSYSTEM_STATES = False

# Réessayer un état arbitraire
test_state = (1,0,0)
subsys_test = pyphi.Subsystem(network, test_state, (0,1,2))
try:
    val_test = pyphi.compute.phi(subsys_test)
    print(f"État {test_state} -> Φ = {val_test:.5f}")
except pyphi.exceptions.StateUnreachableError:
    print("État inaccessible, PyPhi a lancé une exception !")


État inaccessible, PyPhi a lancé une exception !


On peut également calculer \\(\Phi\\) pour d'autres états accessibles:

In [59]:
# Exemple d'états stables pour éviter StateUnreachableError
for s in [(0,0,0), (0,1,1), (1,1,0)]:
    subsys = pyphi.Subsystem(network, s, (0,1,2))
    val = pyphi.compute.phi(subsys)
    print(f"État {s} -> Φ = {val:.5f}")


État (0, 0, 0) -> Φ = 1.87500


État (0, 1, 1) -> Φ = 1.87500


État (1, 1, 0) -> Φ = 1.87500


<a id="sec5"></a>
## 5. Concepts, mécanismes, et structure cause-effet (CES)

PyPhi peut également détailler la structure de cause-effet (CES) :

- Chaque *concept* correspond à un **mécanisme** (ex. nœud simple ou sous-groupe de nœuds) et à son purview (cause et/ou effet).

In [60]:
ces = pyphi.compute.ces(subsystem)
print("Nombre de concepts dans la CES :", len(ces))
print("Liste des mécanismes :", ces.labeled_mechanisms)

Nombre de concepts dans la CES : 3
Liste des mécanismes : (['A', 'B'], ['A', 'C'], ['B', 'C'])


### 5.1. Analyse d’un concept spécifique

Chaque concept est défini par un mécanisme (un ensemble de nœuds) et un purview (un ensemble de nœuds qui subissent ses effets ou le causent). Explorons le premier concept dans la CES, pour comprendre sa cause / effet et son petit phi (\\(\\varphi\\)).


In [61]:
# Prenons le premier concept dans la CES
first_concept = ces[0]
print("Mécanisme:", first_concept.mechanism)
print("Cause purview:", first_concept.cause.purview)
print("Cause repertoire:\n", first_concept.cause.repertoire)
print("Effect purview:", first_concept.effect.purview)
print("Effect repertoire:\n", first_concept.effect.repertoire)
print("Petit phi (φ) =", first_concept.phi)


Mécanisme: (0, 1)
Cause purview: (0, 1, 2)
Cause repertoire:
 [[[0.5 0. ]
  [0.  0. ]]

 [[0.  0. ]
  [0.  0.5]]]
Effect purview: (2,)
Effect repertoire:
 [[[1. 0.]]]
Petit phi (φ) = 0.5


Vous pouvez inspecter chaque concept : 
- son mécanisme,
- son purview, 
- et la valeur \\(\varphi\\ (petit phi) mesurant l’irréductibilité du mécanisme.

---

<a id="sec6"></a>
## 6. Exemple avancé : analyse de causalité et macro-subsystems

### 6.1 Actual Causation

PyPhi offre `pyphi.actual` pour analyser la causalité effective (\"What caused what?\"). On définit un `Transition` : état avant, état après, et on cherche les liens causaux irréductibles.



In [62]:
import pyphi
import pyphi.actual

pyphi.config.VALIDATE_SUBSYSTEM_STATES = False

before_state = (1,1,1)
after_state  = (0,0,0)

transition = pyphi.actual.Transition(
    network,
    before_state,
    after_state,
    cause_indices=(0,1,2),
    effect_indices=(0,1,2)
)

account = pyphi.actual.account(transition)
print("Nombre de causal links :", len(account))
for link in account:
    print(link)


Nombre de causal links : 10
CausalLink
  α = 1.0  [B, C] ◀━━ [A]
CausalLink
  α = 1.0  [A, C] ◀━━ [B]
CausalLink
  α = 1.0  [A, B] ◀━━ [C]
CausalLink
  α = 1.0  [A, B, C] ◀━━ [A, B]
CausalLink
  α = 1.0  [A, B, C] ◀━━ [A, C]
CausalLink
  α = 1.0  [A, B, C] ◀━━ [B, C]
CausalLink
  α = 1.0  [A, B] ━━▶ [C]
CausalLink
  α = 1.0  [A, C] ━━▶ [B]
CausalLink
  α = 1.0  [B, C] ━━▶ [A]
CausalLink
  α = 2.0  [A, B, C] ━━▶ [A, B, C]


### 6.2 Macro-subsystems (coarse-graining, blackboxing)

Le module `pyphi.macro` permet d'agréger des nœuds (coarse-grain) ou de blackboxer certains éléments. 
On peut ainsi recalculer \\(\Phi\\) à une échelle différente, ce qui rejoint l’idée de l’ICT qu’on peut analyser l’information intégrée à divers niveaux.



<a id="sec7"></a>
## 7. Ressources complémentaires

- [Documentation PyPhi](https://pyphi.readthedocs.io/en/stable/)",
- [\"Major Complex\", \"Actual Causation\"](https://github.com/wmayner/pyphi/tree/master/examples)",
- Travaux de Tononi, Koch, Oizumi, Albantakis, etc. (cf. publications IIT)",
- Pour aller plus loin : l’ICT, l’analyse fractale de l’information, etc.

**Fin du notebook**. 
Merci pour votre attention !